In [91]:
#%matplotlib notebook

In [1]:
%matplotlib inline

In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
from netCDF4 import Dataset

In [4]:
import glob

In [28]:
import re

In [148]:
import datetime

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [6]:
def hvel(r):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [7]:
def pres(r):
    return pch+dph*np.exp(-(rmaxh/r)**bh)

In [8]:
def hvel2d(l1,l2,vtx,vty):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

In [9]:
path='../BestTrack/'

In [65]:
data=pd.read_csv(path+'Allstorms.ibtracs_wmo.v03r09.csv', header=1, low_memory=False)

In [66]:
data.head()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,Wind(WMO),Pres(WMO),Center,Wind(WMO) Percentile,Pres(WMO) Percentile,Track_type
0,NaN,Year,#,BB,BB,NaN,YYYY-MM-DD HH:MM:SS,NaN,deg_north,deg_east,kt,mb,NaN,%,%,NaN
1,1848011S09080,1848,02,SI,MM,XXXX848003,1848-01-11 06:00:00,NR,-8.60,79.80,0.0,0.0,reunion,-100.000,-100.000,main
2,1848011S09080,1848,02,SI,MM,XXXX848003,1848-01-12 06:00:00,NR,-9.00,78.90,0.0,0.0,reunion,-100.000,-100.000,main
3,1848011S09080,1848,02,SI,MM,XXXX848003,1848-01-13 06:00:00,NR,-10.40,73.20,0.0,0.0,reunion,-100.000,-100.000,main
4,1848011S09080,1848,02,SI,MM,XXXX848003,1848-01-14 06:00:00,NR,-12.80,69.90,0.0,0.0,reunion,-100.000,-100.000,main


# ANALYSIS STARTS HERE

In [12]:
hurName='TOMAS'
year = '2010'

In [13]:
hurdata = data[data['Name'].str.contains(hurName,na=False) & data['ISO_time'].str.contains(year,na=False)]

In [14]:
hurdata.iloc[0]

Serial_Num                    2010069S12188
Season                                 2010
Num                                      10
Basin                                    SP
Sub_basin                                MM
Name                                  TOMAS
ISO_time                2010-03-10 00:00:00
Nature                                   NR
Latitude                             -12.30
Longitude                           -172.20
Wind(WMO)                              20.0
Pres(WMO)                            1005.0
Center                                 nadi
Wind(WMO) Percentile                  4.063
Pres(WMO) Percentile                  1.388
Track_type                             main
Name: 179026, dtype: object

In [15]:
hurdata.shape

(95, 16)

In [16]:
ntc=hurdata['Basin'].unique()

In [17]:
ntc

array([' SP', ' NA'], dtype=object)

In [18]:
hur1=hurdata.loc[hurdata['Basin']==ntc[1]]

In [19]:
hur1[:13]

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,Wind(WMO),Pres(WMO),Center,Wind(WMO) Percentile,Pres(WMO) Percentile,Track_type
180706,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 06:00:00,TS,9.00,-53.70,30.0,1006.0,atcf,17.564,26.787,main
180707,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 12:00:00,TS,9.80,-55.30,35.0,1003.0,atcf,32.605,41.881,main
180708,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 18:00:00,TS,10.80,-56.80,45.0,998.0,atcf,50.354,57.575,main
180709,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-30 00:00:00,TS,11.90,-57.80,55.0,999.0,atcf,64.336,55.831,main
180710,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-30 06:00:00,TS,12.70,-58.90,60.0,997.0,atcf,69.665,59.549,main
180711,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 09:00:00,TS,13.00,-59.50,60.0,997.0,atcf,69.665,59.549,main
180712,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 12:00:00,TS,13.10,-60.10,65.0,993.0,atcf,73.828,67.865,main
180713,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 18:00:00,TS,13.30,-61.00,80.0,990.0,atcf,85.026,71.117,main
180714,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 20:00:00,TS,13.40,-61.20,85.0,987.0,atcf,87.293,74.957,main
180715,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-31 00:00:00,TS,13.50,-61.70,85.0,982.0,atcf,87.293,80.573,main


In [20]:
plt.plot(hur1['Longitude'],hur1['Latitude'],'o--')
mplleaflet.display()

In [21]:
idf=hur1.iloc[0].Serial_Num

In [22]:
filename=glob.glob('../BestTrack/ibtracs/{}*'.format(idf))

In [23]:
dat=Dataset(filename[0])

In [24]:
dat.variables.keys()

[u'nobs_wmo',
 u'nobs',
 u'genesis_basin',
 u'num_basins',
 u'season',
 u'source',
 u'track_from_source',
 u'name_from_source',
 u'time_wmo',
 u'isotime_wmo',
 u'source_wmo',
 u'averaging_time',
 u'lat_wmo',
 u'lon_wmo',
 u'wind_wmo',
 u'pres_wmo',
 u'basin_wmo',
 u'sub_basin_wmo',
 u'nature_wmo',
 u'dist2land_wmo',
 u'landfall_wmo',
 u'pres_perc_wmo',
 u'wind_perc_wmo',
 u'time',
 u'isotime',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'wind_for_mapping',
 u'pres_for_mapping',
 u'nature_for_mapping',
 u'dist2land',
 u'landfall',
 u'lat_from_source',
 u'lon_from_source',
 u'wind_from_source',
 u'wind_avg_period',
 u'wind_perc_from_source',
 u'pres_from_source',
 u'pres_perc_from_source',
 u'basin',
 u'sub_basin',
 u'pos_stdev',
 u'pos_num',
 u'pos_flag',
 u'wind_quality',
 u'wind_stdev',
 u'wind_condition',
 u'wind_num',
 u'wind_flag',
 u'wind_maximum',
 u'wind_minimum',
 u'wind_median',
 u'pres_quality',
 u'pres_stdev',
 u'pres_condition',
 u'pres_flag',
 u'pres_num',
 u'pres_maximum'

In [25]:
wradii=dat['atcf_wrad'][:13]

In [26]:
isot=dat['isotime'][:13]

In [33]:
isot1=[''.join(x) for x in isot]

In [37]:
time=[re.sub(r'[-: ]+','',x)[:-4] for x in isot1]

In [61]:
lat=dat['atcf_lat'][:13]
lon=dat['atcf_lon'][:13]
pcenter=dat['atcf_pres'][:13]
vmaxh=dat['atcf_wind'][:13]
rmaxh=dat['atcf_rmw'][:13]

In [62]:
dph=pn-pcenter

In [44]:
wradiinp=np.array(wradii)

In [47]:
time

['2010102906',
 '2010102912',
 '2010102918',
 '2010103000',
 '2010103006',
 '2010103012',
 '2010103018',
 '2010103100',
 '2010103106',
 '2010103112',
 '2010103118',
 '2010110100',
 '2010110106']

In [70]:
isot1

['2010-10-29 06:00:00',
 '2010-10-29 12:00:00',
 '2010-10-29 18:00:00',
 '2010-10-30 00:00:00',
 '2010-10-30 06:00:00',
 '2010-10-30 12:00:00',
 '2010-10-30 18:00:00',
 '2010-10-31 00:00:00',
 '2010-10-31 06:00:00',
 '2010-10-31 12:00:00',
 '2010-10-31 18:00:00',
 '2010-11-01 00:00:00',
 '2010-11-01 06:00:00']

In [50]:
w34=wradiinp[:,0,:]

In [53]:
w50=wradiinp[:,1,:]

In [55]:
w64=wradiinp[:,2,:]

In [58]:
w64[:,0]

array([ 0,  0,  0,  0,  0, 20, 20, 30, 30, 30, 15,  0,  0], dtype=int16)

### create inpData file

In [124]:
dic={'t':isot1,'lat':lat,'long':lon,'vmax':vmaxh,'64ne':w64[:,0],'64se':w64[:,1],'64sw':w64[:,2],'64nw':w64[:,3],'50ne':w50[:,0],'50se':w50[:,1],'50sw':w50[:,2],'50nw':w50[:,3],'34ne':w34[:,0],'34se':w34[:,1],'34sw':w34[:,2],'34nw':w34[:,3],'notes':np.zeros(len(time))}

In [125]:
inpData=pd.DataFrame(dic)

In [126]:
inpData['t']=pd.to_datetime(inpData['t'])

In [127]:
inpData['time']=inpData['t']-inpData['t'][0]

In [128]:
inpData['time']=inpData['time'] / pd.Timedelta('1 hour')

In [129]:
inpData=inpData.set_index('t')
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,long,notes,vmax,time
t,,,,,,,,,,,,,,,,,
2010-10-29 06:00:00,0,0,0,0,0,0,0,0,0,0,0,0,9.0,-53.700001,0.0,30,0.0
2010-10-29 12:00:00,75,75,40,0,0,0,0,0,0,0,0,0,9.8,-55.299999,0.0,35,6.0
2010-10-29 18:00:00,90,90,50,50,0,0,0,0,0,0,0,0,10.8,-56.799999,0.0,45,12.0
2010-10-30 00:00:00,90,90,50,50,30,30,0,0,0,0,0,0,11.9,-57.799999,0.0,55,18.0
2010-10-30 06:00:00,90,90,50,50,30,30,0,0,0,0,0,0,12.7,-58.900002,0.0,60,24.0
2010-10-30 12:00:00,120,90,60,50,50,40,30,20,20,0,20,0,13.1,-60.099998,0.0,65,30.0
2010-10-30 18:00:00,120,90,60,50,50,40,30,20,20,0,20,0,13.3,-61.000000,0.0,80,36.0
2010-10-31 00:00:00,120,90,60,50,50,40,30,20,30,20,20,10,13.5,-61.700001,0.0,85,42.0
2010-10-31 06:00:00,120,90,60,50,60,50,30,20,30,20,20,10,13.8,-62.400002,0.0,85,48.0


In [157]:
column_order=['time','lat','long','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw','notes']

In [158]:
inpData.to_csv('tmp/inpData.txt',index=False, columns=column_order, sep='\t')

### create bulInfo.txt file

In [138]:
tt=pd.to_datetime(isot1[0])

In [152]:
tt=datetime.datetime.strftime(tt,'%d %b %Y %H:%M:%S')

In [153]:
dic0={'advNo':[1],'tShift':[0],'$date':tt,'land':[1],'notes':[0]}

In [154]:
bul=pd.DataFrame.from_dict(dic0)

In [155]:
bul

,$date,advNo,land,notes,tShift
0,29 Oct 2010 06:00:00,1,1,0,0


In [156]:
bul.to_csv('tmp/bulInfo.txt',index=False, columns=['advNo','tShift','$date','land','notes'], sep='\t')

### create info.pr file

## compute b,k

In [46]:
rh=np.linspace(0.,5*rmaxh,100)

In [47]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,hvel(rh))
plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')

NameError: global name 'kh' is not defined

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(rh/1000,pres(rh)/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure [PA]',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:
ni,nj=1000,1000
xl=np.linspace(-maxR, maxR,ni+1)
yl=np.linspace(-maxR, maxR,nj+1)
q1,q2=np.meshgrid(xl,yl)

In [ ]:
zx,zy, pr=hvel2d(q1,q2,vtx,vty)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,pr/100,20)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Pressure contours')


plt.colorbar(label='PA')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,pr[500,:]/100)

plt.xlabel('km',fontsize=18)
plt.ylabel('Pressure',fontsize=18)
plt.title('Pressure vs radius')


plt.show()

In [ ]:

vel=np.sqrt(zx**2+zy**2)

In [ ]:
plt.figure(figsize=(10,8))
plt.contourf(q1/1000,q2/1000,vel,20)
plt.streamplot(q1/1000,q2/1000,zx,zy, color=vel, linewidth=0.1*vel)
plt.xlabel('km',fontsize=18)
plt.ylabel('Km',fontsize=18)
plt.title('Velocity vs radius')

plt.colorbar(label='m/s')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(q1[0,:]/1000,vel[ni/2,:],label='horizontal')
plt.plot(q2[:,0]/1000,vel[:,nj/2],label='verical')

plt.xlabel('km',fontsize=18)
plt.ylabel('Velocity [m/s]',fontsize=18)
plt.title('Velocity vs radius')


plt.legend(loc=0)
plt.show()

In [ ]:
np.savetxt('tmp/phol',[q1[0,:],pr[500,:]])
np.savetxt('tmp/uhol',[q1[0,:],vel[ni/2,:],vel[:,nj/2]])

In [ ]:
pd.DataFrame({'Min Pressure':[pch/100],'Max Velocity':[vmaxh]})